# Before you start:
This notebook shows how to execute YOLOv7 to detect loading bay door positions in video. The data for this project
was annotated using RoboFlow. 

The data can be accessed here: curl -L "https://app.roboflow.com/ds/vuDHqfzW8O?key=1R86pjvC9o" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


Miscellaneous notes:
- Go to "Train" to train a model from scratch, "Test" to assess the quality of a trained model, and "Detect" to run detection on a sample. 
- To access the training data & pretrained model for this repo, run the following cell. You will then need to navigate to the "Helpers" section, and follow the instructions there to set up your code for YOLOv7. 

In [ ]:
!curl -L "https://app.roboflow.com/ds/3RUpGzlTpl?key=o1uRVtIeRj" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
!pip install -r requirements.txt
!pip install setuptools==59.5.0

## Helpers

To set up your RoboFlow annotated data for YOLO, we've provided some helper functions to quickly clean up the filenames. 
Simply run all the cells below

In [3]:
! mkdir v-test
! mv train/ v-test/
! mkdir v-test/test/ 
! mkdir v-test/test/labels
! mkdir v-test/test/images
! mkdir v-test/valid/ 
! mkdir v-test/valid/images
! mkdir v-test/valid/labels

In [ ]:
import os 

# remove roboflow extra junk

count = 0
for i in sorted(os.listdir('v-test/train/labels')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = 'v-test/train/labels/'+i
    dest = 'v-test/train/labels/'+j[0]+dict1[count]+'.txt'
    os.rename(source, dest)
    
count = 0
for i in sorted(os.listdir('v-test/train/images')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = 'v-test/train/images/'+i
    dest = 'v-test/train/images/'+j[0]+dict1[count]+'.jpg'
    os.rename(source, dest)

In [ ]:
# Set up validation set

import os
import shutil
 
source = 'v-test/train/images/'
destination = 'v-test/valid/images/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
# print(type(f))
# print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) >=420:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('ee')[0] == 'garag' and int(f[7:-5]) <= 150:
        # print(int(f[7:-5]))
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('-')[0]=='garagesnap' and int(f[11:16]) < 150:
        # print(int(f[11:16]))
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

    
source = 'v-test/train/labels/'
destination = 'v-test/valid/labels/'
 
allfiles = os.listdir(source)
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage' and int(f[7:-5]) >=463:
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('ee')[0] == 'garag' and int(f[7:-5]) <= 150:
        # print(int(f[7:-5]))
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)
    if f.split('-')[0]=='garagesnap' and int(f[11:16]) < 150:
        print(int(f[11:16]))
        print(f)
        print(source + f)
        print(destination +f)
        # break
        shutil.move(source + f, destination + f)

In [ ]:
import os
import shutil
 
source = 'v-test/train/images/'
destination = 'v-test/test/images/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage':
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

source = 'v-test/train/labels/'
destination = 'v-test/test/labels/'
 
allfiles = os.listdir(source)
 
for f in allfiles:
    # print(type(f))
    # print(f.split
    if f.split('c')[0] == 'garage':
        # print(f[7:-5])
        # print(f)
        # print(source + f)
        # print(destination +f)
        # break
        shutil.move(source + f, destination + f)

## Train


This section shows how to fine tune a model for a custom dataset.

### Training instructions & tips
- If you need to change the locations of your training/validation/test images, then be sure to go to custom.yaml in the "data" folder, and change the path locations. 
- also in custom.yaml, you can set and label the number of labels you want to be able to detect with your model
- If you are on a distributed machine, use the second train command in the cell below. Be sure to change "nproc_per_node" to accurately reflect the number of GPUs on your device. 
- use the hyp.scratch.custom.yaml file to change hyperparameters for training. 

In [ ]:
# !python train.py --workers 8 --device 0 --batch-size 8 --data data/coco.yaml --img 1280 720 --cfg cfg/training/yolov7x.yaml --weights yolov7_training.pt --name yolov7-ballhandler --hyp data/hyp.scratch.custom.yaml --epochs 50

!python -m torch.distributed.launch --nproc_per_node 2 --master_port 9527 train.py --workers 16 --device 0,1 --sync-bn --batch-size 8 --data data/coco.yaml --img 1280 720 --cfg cfg/training/yolov7.yaml --weights yolov7_training.pt --name yolov7-ballhandler --hyp data/hyp.scratch.custom.yaml --epochs 50


## Detect

Use the following cell to run detection on a submitted image. 

- Change image or video being detected on using --source tag
- img size X dimension must be correct for this to run. no Y needed
- if you want to do detection on anything other than the test set, you will need to upload the video to the platform using a standard file upload in the top left corner. 

In [ ]:
# get test video: 
!pip install gdown
!gdown https://drive.google.com/uc?id=1hUk6mdIXwOzcSGYMpWjJR6ngCgzVgTkB

In [ ]:
!python detect.py --weights runs/train/yolov7-ballhandler/weights/best.pt --conf 0.25 --img-size 1280 --source "test-video-final.mp4" --name test


## Testing

In [ ]:
!python test.py --data data/test.yaml --img 1280 --batch 16 --conf 0.001 --iou 0.65 --device 0 --weights runs/train/yolov7-ballhandler/weights/best.pt --name yolov7_ballhandler_testing
